In [1]:
import torch
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import PIL.Image as Image
import transforms as T
from engine import train_one_epoch, evaluate
import utils
from MathExpressionDataset import MEdataset
import os


In [2]:

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [3]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 109 #108 LaTeX symbols + the background/nothing

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [4]:
trainDir = r'C:\Users\maxwe\Desktop\My Documents\MathExprSolverMx\MathExprSolverMx\AidaCalculusHandWrittenMathDataset\archive\train'
testDir = r'C:\Users\maxwe\Desktop\My Documents\MathExprSolverMx\MathExprSolverMx\AidaCalculusHandWrittenMathDataset\archive\test'


In [5]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# use our dataset and defined transformations
dataset = MEdataset(trainDir, get_transform(train=True))
dataset_test = MEdataset(testDir, get_transform(train=False))

# split the dataset in train and test set
indicesTrain = torch.randperm(len(dataset)).tolist()
indicesTest = torch.randperm(len(dataset_test)).tolist()
dataset = torch.utils.data.Subset(dataset, indicesTrain[:])
dataset_test = torch.utils.data.Subset(dataset_test, indicesTest[:])


In [6]:
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [8]:
# move model to the right device
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [9]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [10]:
# let's train it for 10 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [    0/49500]  eta: 2 days, 7:58:58  lr: 0.000010  loss: 7.7644 (7.7644)  loss_classifier: 4.2789 (4.2789)  loss_box_reg: 0.8754 (0.8754)  loss_objectness: 2.2708 (2.2708)  loss_rpn_box_reg: 0.3393 (0.3393)  time: 4.0715  data: 2.6382  max mem: 1185
Epoch: [0]  [   10/49500]  eta: 10:26:15  lr: 0.000060  loss: 7.0558 (6.9855)  loss_classifier: 4.2014 (4.1633)  loss_box_reg: 0.8815 (0.9074)  loss_objectness: 1.6610 (1.5891)  loss_rpn_box_reg: 0.3393 (0.3257)  time: 0.7593  data: 0.2413  max mem: 2831
Epoch: [0]  [   20/49500]  eta: 8:11:17  lr: 0.000110  loss: 5.7429 (6.0621)  loss_classifier: 3.8078 (3.7705)  loss_box_reg: 0.9329 (0.9188)  loss_objectness: 0.6193 (0.9823)  loss_rpn_box_reg: 0.3326 (0.3905)  time: 0.4220  data: 0.0015  max mem: 2831
Epoch: [0]  [   30/49500]  eta: 7:17:35  lr: 0.000160  loss: 3.9489 (5.2330)  loss_classifier: 2.4680 (3.1743)  loss_box_reg: 0.9341 (0.9181)  loss_objectness: 0.0922 (0.7022)  loss_rpn_box_reg: 0.5289 (0.4384)  time: 0.4050  dat

In [10]:

savePath = os.path.join(os.getcwd(), 'model1.pt')
torch.save(model.state_dict(), savePath)